In [1]:
import picaso.opacity_factory as opa_fa
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
from virga import justdoit as vj
from virga import justplotit as cldplt
jpi.output_notebook()

wave_range = [0.45,1.8]

opa = jdi.opannection(wave_range=wave_range)

Loading BokehJS ...

In [2]:
# Warm jup:
wj = jdi.inputs()
wj.phase_angle(0)

## Selecing one from the GMagAO-X target list:

# HD 143761 b:
radius = 11.34 #Rearth
radius = radius*u.Rearth.to(u.Rjup)
sep = 0.23 #au
msinie = 349.613 #Mjup
Teq = 608.1

# Star:
StarTeff = 5777.0
wj.star(opa, StarTeff, 0, 4.0, semi_major = sep, semi_major_unit=jdi.u.Unit('au'))

wj.gravity(radius=radius, radius_unit=jdi.u.Unit('R_jup'), 
            mass = msinie, mass_unit=jdi.u.Unit('M_earth'))

#cj.atmosphere()
wj.guillot_pt(Teq)

co=1.0 #c/o ratio 1x solar
logfeh = 0.5 #10^0.5 ~ 3x solar
wj.chemeq_visscher(co,logfeh) #adds in template chemistry from sonora
wj_df = wj.spectrum(opa, calculation='reflected light', full_output = True)

/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/atmsetup.py:190: UserWarning: Ignoring graphite in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)


In [3]:
cj = jdi.inputs()
cj.phase_angle(0)

## Selecing one from the GMagAO-X target list:

# HD 183263 b:
radius = 11.96 #Rearth
radius = radius*u.Rearth.to(u.Rjup)
sep = 1.508 #au
msinij = 3.704 #Mjup
Teq = 239.708

# Star:
StarTeff = 5875.99
cj.star(opa, StarTeff, 0, 4.0, semi_major = sep, semi_major_unit=jdi.u.Unit('au'))

cj.gravity(radius=radius, radius_unit=jdi.u.Unit('R_jup'), 
            mass = msinij, mass_unit=jdi.u.Unit('M_jup'))

#cj.atmosphere()
cj.guillot_pt(Teq)

co=1.0 #c/o ratio 1x solar
logfeh = 0.5 #10^0.5 ~ 3x solar
cj.chemeq_visscher(co,logfeh) #adds in template chemistry from sonora
cj_df = cj.spectrum(opa, calculation='reflected light', full_output=True)



/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/atmsetup.py:190: UserWarning: Ignoring graphite in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)


In [4]:
out = jdi.get_contribution(cj, opa, at_tau=1)

jpi.show(jpi.molecule_contribution(out, opa, plot_width=700, x_axis_type='log'))

/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/atmsetup.py:190: UserWarning: Ignoring graphite in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)


In [5]:
outw = jdi.get_contribution(wj, opa, at_tau=1)

jpi.show(jpi.molecule_contribution(outw, opa, plot_width=700, x_axis_type='log'))

/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/atmsetup.py:190: UserWarning: Ignoring graphite in input file, not a recognized molecule
  warnings.warn("Ignoring %s in input file, not a recognized molecule" % i, UserWarning)


# Add Climate

In [15]:
# Planet metallicity/C to O:
mh = '+100'#'+1.0' #log metallicity
CtoO = '100'#'1.0' # CtoO ratio

# Load opacities:
ck_db = f'/Volumes/Oy/picaso/reference/kcoeff_2020/sonora_2020_feh{mh}_co_{CtoO}.data.196'
ck_db
# Notice The keyword ck is set to True because you want to use the correlated-k opacities for your calculation
# and not the line by line opacities
opacity_ck = jdi.opannection(ck = True, ck_db=ck_db) # grab your opacities

In [25]:
# Warm jup:
cl_wj = jdi.inputs(calculation="planet", climate = True)
tint = 150

cl_wj.effective_temp(tint)

# HD 143761 b:
radius = 11.34 #Rearth
radius = radius*u.Rearth.to(u.Rjup)
msinie = 349.613 #Mjup

cl_wj.gravity(radius=radius, radius_unit=jdi.u.Unit('R_jup'), 
            mass = msinie, mass_unit=jdi.u.Unit('M_earth'))

cl_wj.phase_angle(np.radians(0))

# Star:
StarTeff = 5777.0
metallicity = 0
logg = 4.0
StarRadius = 1 #u.Rsun

sep = 0.23 #au

cl_wj.star(opa, StarTeff, metallicity, logg, semi_major = sep, semi_major_unit=jdi.u.Unit('au'))#,
          #radius = StarRadius, radius_unit = jdi.u.Unit('Rsun'))

# PT profile guess:
nlevels = 90 #number of plane-parallel levels
Teq = 608.1 #eq temp from sep/radius
p_bottom = 2 #log pressure at bottom of atm calc
p_top = -6 #log pressure at top of atm calc


pt = cl_wj.guillot_pt(Teq, nlevel=nlevels, T_int = tint, p_bottom=p_bottom, p_top=-6)
temp_guess = pt['temperature'].values
pressure_guess = pt['pressure'].values



In [26]:
nofczns = 1 #number of convective zones

nstr_upper = 85 #top most level of guessed convective zone
nstr_deep = -2 # this is always the case always use this number
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convection zones

rfacv = 0.5

#Input climate:
cl_wj.inputs_climate(temp_guess= temp_guess, pressure= pressure_guess,
                      nstr = nstr, nofczns = nofczns , rfacv = rfacv)
# Run climate run:
out = cl_wj.climate(opacity_ck, save_all_profiles=True, with_spec=True)

/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/justdoit.py:3789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'nostar' in self.inputs['star'].values():
/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/optics.py:2136: RuntimeWarning: divide by zero encountered in log
  ((t_interp)   * np.log(data_high[i+'_'+str(jhigh)])))
/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/optics.py:2135: RuntimeWarning: divide by zero encountered in log
  ln_kappa = np.exp(((1-t_interp) * np.log(data_low[i+'_'+str(jlow)]) ) +
/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/climate.py:412: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'gauss_wts' of function 'climate'.

For more information visit https://numba.readthedocs.io/en/s

ValueError: unable to broadcast argument 3 to output array
File "/Users/loganpearce/anaconda3/envs/py39/lib/python3.9/site-packages/picaso/climate.py", line 1, 